### Analysis of U.S. Total Energy Consumptions Over Time
The annual total energy data from the U.S. Energy Information Administration (EIA)
- Time period: 1949 - 2024
  - Note: Data on 3 features regarding energy xpenditures are available from 1970 to 2023.
- Purposes:
  - Exploratory Data Analysis (EDA) with visualization: 
    + Visualize patterns of U.S. energy consumptions over time: total end-use energy and electricity
    + Display energy consumptions by end-use sector: Residential, Commercial, Industrial, and Transportation 
    + Display relationship between energy & electricity consumptions and GDP & population; changes in the energy intensity indicators (per GDP and per capita), energy expenditures, and average energy prices
    + Display relationship between energy & electricity consumptions and total degree days (= Heating Degree Days (HDD) + Cooling Degree Days (CDD)). 
  
  - Predictive regression analysis: 
    Predict the consumptions in total end-use energy and electricity (DVs) with the predictors of GDP, population, and total degree days (IVs)
    + Year 1949-2024: Explore the regression relationships between DVs and IVs, using OLS
    + Year 2025-2030: Forcate the consumptions in total end-use energy and electricity (DVs) with GPD and population (* Note: Duo to the very high correlation between GDP and popution (r=0.985, multicollinear issue), the final predict model can include population as the only IV.) 

In [1]:
# import sys
# import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px     # for interactive plots
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [2]:
# Load the dataset - US-level energy data
df1_energy = pd.read_csv('USdata4_all_2ForAnalysis_1949-2024.csv')

# Set display width for better readability  
pd.set_option('display.width', 600)  

# Data information
display(df1_energy.info())
print(df1_energy.head(2), "\n")

# # Check for missing values
# missing_values = df1_energy.isnull().sum()
# print("Missing values in each column:\n", missing_values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Year                            76 non-null     int64  
 1   TotPrimEnergy_1Production       76 non-null     float64
 2   TotPrimEnergy_2Consumed         76 non-null     float64
 3   TotEnergy_2EndUse               76 non-null     float64
 4   EnergyUse1_Residential          76 non-null     float64
 5   EnergyUse2_Commercial           76 non-null     float64
 6   EnergyUse3_Industrial           76 non-null     float64
 7   EnergyUse4_Transportation       76 non-null     float64
 8   TotElectricity_2Used            76 non-null     float64
 9   ElectricityUse1_Residential     76 non-null     float64
 10  ElectricityUse2_Commercial      76 non-null     float64
 11  ElectricityUse3_Industrial      76 non-null     float64
 12  ElectricityUse4_Transportation  76 non

None

   Year  TotPrimEnergy_1Production  TotPrimEnergy_2Consumed  TotEnergy_2EndUse  EnergyUse1_Residential  EnergyUse2_Commercial  EnergyUse3_Industrial  EnergyUse4_Transportation  TotElectricity_2Used  ElectricityUse1_Residential  ...  GDP_Nominal  GDP_Real  POP_US  Energy_Expenditure  pct_EnergyExpend_w_GDP  EnergyExpend_per_capita  PrimEnergyUse_per_GDP  PrimEnergyUse_per_capita  days_cooling  days_heating
0  1949                  30613.106                30866.419          28438.470                4688.482               2869.013              12979.350                   7901.625               868.393                      227.894  ...        272.5    2261.9   149.2                 NaN                     NaN                      NaN                  13.65                       207          1103          4933
1  1950                  34459.730                33527.374          30861.148                5075.876               3059.238              14319.447                   8406.588       

#### Exploratory Data Analysis (EDA)

In [25]:
# Total Primary Energy Production and Energy Consumptions: 1949-2024
# - Total primary energy production 
# - Total primary energy consumption
# - End-use energy consumption
# - Electricity consumption (end-use)
# The energy units are in Trillion Btu (British thermal units).

# Create interactive line chart 
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["TotPrimEnergy_1Production"],
    mode="lines", name="Primary Energy Production"
))
fig1.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["TotPrimEnergy_2Consumed"],
    mode="lines", name="Primary Energy Consumption"
))
fig1.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["TotEnergy_2EndUse"],
    mode="lines", name="End-Use Energy Consumption"
))
fig1.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["TotElectricity_2Used"],
    mode="lines", name="Electricity End-Use"
))

# Add a vertical line at Year = 1958, when Consumption > Production
fig1.add_vline(
    x=1958,
    line_width=2,
    line_dash="dash",
    line_color="darkred",
    # annotation_text="Year 1958: Consumption > Production",
    # annotation_position="top right"
)
# Add annotation manually
fig1.add_annotation(
    x=1958, y=df1_energy["TotPrimEnergy_1Production"].max() / 3,  # 1/3 of y-axis
    text=" Year 1958: Consumption > Production",
    showarrow=False,
    xanchor="left",    # text appears to the right of the line
    font=dict(color="darkred", size=12, family="Arial")  # customize color, size, font
)

# Add a vertical line at Year = 2019, when Production > Consumption
fig1.add_vline(
    x=2019,
    line_width=2,
    line_dash="dash",
    line_color="darkred",
    annotation_text="Year 2019: Production > Consumption ",
    annotation_position="top left",
    annotation_font=dict(color="darkred", size=12, family="Arial")  # customize color, size, font
)

# Layout with custom size
fig1.update_layout(
    title="Total Primary Energy Production and Energy Consumptions 1949–2024",
    xaxis_title="Year",
    yaxis_title="Energy (Trillion Btu)",
    template="plotly_white",
    hovermode="x unified",
    width=900,   # adjust width
    height=500,   # adjust height
    xaxis=dict(showgrid=True,
        gridcolor="lightgray",  # darker than default
        gridwidth=1),
    yaxis=dict(showgrid=True,
        gridcolor="lightgray",  # darker than default
        gridwidth=1)
)

fig1.show()
# Export interactive HTML
fig1.write_html("US_Energy_Production_Consumption.html")

In [ ]:
# Total end-use energy consumption by sector 1949-2024

df_sectors = df1_energy[[
    "Year",
    "EnergyUse1_Residential",
    "EnergyUse2_Commercial",
    "EnergyUse3_Industrial",
    "EnergyUse4_Transportation"
]]

# Melt for plotting
df_sectors_melt = df_sectors.melt(id_vars="Year", 
                                  var_name="End-Use Sector", 
                                  value_name="Energy Consumption (Trillion Btu)")
fig2 = px.bar(
    df_sectors_melt,
    x="Year",
    y="Energy Consumption (Trillion Btu)",
    color="End-Use Sector",
    title="U.S. End-Use Energy Consumption by Sector 1949–2024",
    barmode="stack"
)

fig2.update_layout(
    template="plotly_white",
    width=900,
    height=500
)

fig2.show()

In [ ]:
# Total end-use energy consumption by sector 1949-2024
# Notes: Notable impacts on energy consumption
# -- 1982-1983: Global Debt Crisis
# -- 2008-2009: Financial Crisis
# -- 2020: COVID-19 Pandemic

fig2_b = go.Figure()
# Add total
fig2_b.add_trace(go.Scatter(x=df1_energy["Year"], y=df1_energy["TotEnergy_2EndUse"],
                            mode="lines", name="Total End-Use", line=dict(width=3, color="black")))

# Add each sector
for col, color in zip(
    ["EnergyUse1_Residential", "EnergyUse2_Commercial", "EnergyUse3_Industrial", "EnergyUse4_Transportation"],
    ["blue", "red", "green", "purple"]
):
    fig2_b.add_trace(go.Scatter(x=df1_energy["Year"], y=df1_energy[col],
                             mode="lines", name=col, line=dict(width=2, color=color)))

# Add a vertical line at Year=1983, Global Debt Crisis
fig2_b.add_vline(
    x=1983,
    line_width=2,
    line_dash="dash",
    line_color="darkred",
    annotation_text="Y1982-1983: Global Debt Crisis ",
    annotation_position="top left",
    annotation_font=dict(color="darkred", size=12, family="Arial")  # customize color, size, font
)

# Add a vertical line at Year=2009, 2008 Financial Crisis
fig2_b.add_vline(
    x=2009,
    line_width=2,
    line_dash="dash",
    line_color="darkred",
    # annotation_text="Y2008: Financial Crisis  ",
    # annotation_position="top left",
    # annotation_font=dict(color="darkred", size=12, family="Arial")  # customize color, size, font
)
# Add annotation manually
fig2_b.add_annotation(
    x=2009, y=df1_energy["TotEnergy_2EndUse"].max() * 2/3,  # 2/3 of y-axis
    text="Y2008: Financial Crisis ",
    showarrow=False,
    xanchor="right",    # text appears to the left of the line
    font=dict(color="darkred", size=12, family="Arial")  # customize color, size, font
)

# Add a vertical line at Year=2020, COVID pandemic impact
fig2_b.add_vline(
    x=2020,
    line_width=2,
    line_dash="dash",
    line_color="darkred",
    annotation_text="Y2020: COVID Pandemic Impact  ",
    annotation_position="top left",
    annotation_font=dict(color="darkred", size=12, family="Arial")  # customize color, size, font
)

# Layout update
fig2_b.update_layout(
    title="U.S. End-Use Energy Consumption by Sector 1949–2024", 
    xaxis_title="Year",
    yaxis_title="Energy Consumption (Trillion Btu)",
    template="plotly_white",
    width=900,
    height=500,
    hovermode="x unified" 
)

fig2_b.show()

In [ ]:
# Total electricity consumption by sector (end-use) 1949-2024
df_sectors_elec = df1_energy[[
    "Year",
    "ElectricityUse1_Residential",
    "ElectricityUse2_Commercial",
    "ElectricityUse3_Industrial",
    "ElectricityUse4_Transportation"
]]

# Melt for plotting
df_sectors_melt = df_sectors_elec.melt(id_vars="Year", 
                                  var_name="End-Use Sector", 
                                  value_name="Electricity (Trillion Btu)")
fig3 = px.bar(
    df_sectors_melt,
    x="Year",
    y="Electricity (Trillion Btu)",
    color="End-Use Sector",
    title="U.S. Electricity Consumption by Sector (End-Use) 1949–2024",
    barmode="stack"
)

fig3.update_layout(
    template="plotly_white",
    width=900,
    height=500
)

fig3.show()


In [99]:
# Total electricity consumption by sector (end-use) 1949-2024
fig3_b = go.Figure()
# Add total
fig3_b.add_trace(go.Scatter(x=df1_energy["Year"], y=df1_energy["TotElectricity_2Used"],
                            mode="lines", name="Total Electricity End-Use", line=dict(width=3, color="black")))

# Add each sector
for col, color in zip(
    ["ElectricityUse1_Residential", "ElectricityUse2_Commercial", "ElectricityUse3_Industrial", "ElectricityUse4_Transportation"],
    ["blue", "red", "green", "purple"]
):
    fig3_b.add_trace(go.Scatter(x=df1_energy["Year"], y=df1_energy[col],
                             mode="lines", name=col, line=dict(width=2, color=color)))

# Add a vertical line at Year=1983, Global Debt Crisis
fig3_b.add_vline(
    x=1983,
    line_width=2,
    line_dash="dash",
    line_color="darkred",
    annotation_text="Y1982-1983: Global Debt Crisis ",
    annotation_position="top left",
    annotation_font=dict(color="darkred", size=12, family="Arial")  # customize color, size, font
)

# Add a vertical line at Year=2009, 2008 Financial Crisis
fig3_b.add_vline(
    x=2009,
    line_width=2,
    line_dash="dash",
    line_color="darkred",
    # annotation_text="Y2008: Financial Crisis  ",
    # annotation_position="top left",
    # annotation_font=dict(color="darkred", size=12, family="Arial")  # customize color, size, font
)
# Add annotation manually
fig3_b.add_annotation(
    x=2009, y=df1_energy["TotElectricity_2Used"].max() * 2/3,  # 2/3 of y-axis
    text="Y2008: Financial Crisis ",
    showarrow=False,
    xanchor="right",    # text appears to the left of the line
    font=dict(color="darkred", size=12, family="Arial")  # customize color, size, font
)

# Add a vertical line at Year=2020, COVID pandemic impact
fig3_b.add_vline(
    x=2020,
    line_width=2,
    line_dash="dash",
    line_color="darkred",
    annotation_text="Y2020: COVID Pandemic Impact  ",
    annotation_position="top left",
    annotation_font=dict(color="darkred", size=12, family="Arial")  # customize color, size, font
)

# Layout update
fig3_b.update_layout(
    title="U.S. Electricity Consumption by Sector (End-Use) 1949–2024", 
    xaxis_title="Year",
    yaxis_title="Electricity (Trillion Btu)",
    template="plotly_white",
    width=900,
    height=500,
    hovermode="x unified" 
)

fig3_b.show()

In [ ]:
# Relationship between total energy consumption and real GDP
# - End-use energy consumption (Trillion Btu)
# - Real Gross Domestic Product (GDP) in chained 2017 dollars

# Scatter plot: Total Energy Consumption vs Real GDP
fig = px.scatter(
    df1_energy,
    x="GDP_Real",               # Real GDP
    y="TotEnergy_2EndUse",      # Total end-use energy consumption
    size="POP_US",              # optional: bubble size by population
    color="Year",               # optional: color points by year
    color_continuous_scale=px.colors.sequential.Viridis,
    title="Total Energy Consumption vs. Real GDP 1949–2024",
    labels={
        "GDP_Real": "Real GDP (Billion chained (2017) dollars)",
        "TotEnergy_2EndUse": "Energy Consumption (Trillion Btu)"
    },
    hover_data=["Year", "POP_US"] # show extra info on hover
)

# Optional: add trendline using statsmodels (regression) DarkSlateGrey
# fig.update_traces(marker=dict(size=8, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    template="plotly_white",
    width=900,
    height=500
)

fig.show()


In [4]:
# Relationship between total energy consumption, electricity consumption, and real GDP
# - End-use energy consumption (Trillion Btu)
# - Electricity consumption (end-use, Trillion Btu)
# - Real Gross Domestic Product (GDP, Billion chained (2017) dollars)

fig = go.Figure()

# Scatter for Total End-Use Energy
fig.add_trace(go.Scatter(
    x=df1_energy["GDP_Real"],
    y=df1_energy["TotEnergy_2EndUse"],
    mode="markers",
    name="Total End-Use Energy",
    marker=dict(size=8, color="blue", opacity=0.7),
    text=df1_energy["Year"],
    hovertemplate="Year: %{text}<br>GDP: %{x}<br>Total Energy: %{y} Trillion Btu<extra></extra>"
))

# Scatter for Electricity End-Use
fig.add_trace(go.Scatter(
    x=df1_energy["GDP_Real"],
    y=df1_energy["TotElectricity_2Used"],
    mode="markers",
    name="Electricity",
    marker=dict(size=8, color="orange", opacity=0.7),
    text=df1_energy["Year"],
    hovertemplate="Year: %{text}<br>GDP: %{x}<br>Electricity: %{y} Trillion Btu<extra></extra>"
))

# Layout
fig.update_layout(
    title=dict(
        text="Real GDP vs Total Energy and Electricity Consumption (1949–2024)",
        x=0.5, xanchor="center"
    ),
    xaxis_title="Real GDP (Billion chained 2017 $)",
    yaxis_title="Usage (Trillion Btu)",
    template="plotly_white",
    width=900,
    height=500,
    hovermode="closest",
    legend=dict(orientation="h", x=0.5, xanchor="center", y=1.1)
)

fig.show()


In [5]:
# Relationship between total energy consumption, electricity consumption, and population
# - End-use energy consumption (Trillion Btu)
# - Electricity consumption (end-use, Trillion Btu)
# - Total Resident Population (Million people)

fig = go.Figure()

# Scatter for Total End-Use Energy
fig.add_trace(go.Scatter(
    x=df1_energy["POP_US"],
    y=df1_energy["TotEnergy_2EndUse"],
    mode="markers",
    name="Total End-Use Energy",
    marker=dict(size=8, color="blue", opacity=0.7),
    text=df1_energy["Year"],
    hovertemplate="Year: %{text}<br>GDP: %{x}<br>Total Energy: %{y} Trillion Btu<extra></extra>"
))

# Scatter for Electricity End-Use
fig.add_trace(go.Scatter(
    x=df1_energy["POP_US"],
    y=df1_energy["TotElectricity_2Used"],
    mode="markers",
    name="Electricity",
    marker=dict(size=8, color="orange", opacity=0.7),
    text=df1_energy["Year"],
    hovertemplate="Year: %{text}<br>GDP: %{x}<br>Electricity: %{y} Trillion Btu<extra></extra>"
))

# Layout
fig.update_layout(
    title=dict(
        text="Population vs Total Energy and Electricity Consumption (1949–2024)",
        x=0.5, xanchor="center"
    ),
    xaxis_title="Total Population (Million people)",
    yaxis_title="Usage (Trillion Btu)",
    template="plotly_white",
    width=900,
    height=500,
    hovermode="closest",
    legend=dict(orientation="h", x=0.5, xanchor="center", y=1.1)
)

fig.show()

In [15]:
# Relationship between energy usage and real GDP and population
# - End-use energy consumption (Trillion Btu)
# - Electricity consumption (end-use, Trillion Btu)
# - Real GDP (Billion chained (2017) dollars)
# - Total Resident Population (Million people)

# Create subplot grid: 1 row x 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        "Energy & Electricity vs Real-GDP",
        "Energy & Electricity vs Population"
    )
)

# --- Left Plot: Real GDP ---
fig.add_trace(go.Scatter(
    x=df1_energy["GDP_Real"], y=df1_energy["TotEnergy_2EndUse"],
    mode="markers+lines", name="Total Energy", marker=dict(color="blue")
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df1_energy["GDP_Real"], y=df1_energy["TotElectricity_2Used"],
    mode="markers+lines", name="Electricity", marker=dict(color="orange")
), row=1, col=1)

# --- Right Plot: Population ---
fig.add_trace(go.Scatter(
    x=df1_energy["POP_US"], y=df1_energy["TotEnergy_2EndUse"],
    mode="markers+lines", name="Total Energy", marker=dict(color="blue"), showlegend=False
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=df1_energy["POP_US"], y=df1_energy["TotElectricity_2Used"],
    mode="markers+lines", name="Electricity", marker=dict(color="orange"), showlegend=False
), row=1, col=2)

# Layout
fig.update_layout(
    title=dict(
        text="U.S. Energy & Electricity vs GDP and Population (1949–2024)",
        x=0.5, xanchor="center"
    ),
    template="plotly_white",
    width=900, height=500,
    legend=dict(
        orientation="h", y=-0.2, x=0.5, xanchor="center", yanchor="top"
    )
)

# Axis labels
fig.update_xaxes(title_text="Real GDP (Billion chained 2017 dollars)", row=1, col=1)
fig.update_yaxes(title_text="Usage (Trillion Btu)", row=1, col=1)

fig.update_xaxes(title_text="Population (Millions)", row=1, col=2)
fig.update_yaxes(title_text="Usage (Trillion Btu)", row=1, col=2)

fig.show()


In [13]:
# Relationship between energy usage and real GDP and population
# - End-use energy consumption (Trillion Btu)
# - Electricity consumption (end-use, Trillion Btu)
# - Real GDP (Billion chained (2017) dollars)
# - Total Resident Population (Million people)

# Create subplot grid: 2 row x 2 columns
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Total Energy vs Real-GDP",
        "Total Energy vs Population",
        "Electricity vs Real-GDP",
        "Electricity vs Population"
    )
)

# --- Top Left: Energy vs GDP ---
fig.add_trace(go.Scatter(
    x=df1_energy["GDP_Real"], y=df1_energy["TotEnergy_2EndUse"],
    mode="markers+lines", name="Total Energy", marker=dict(color="blue")
), row=1, col=1)
# --- Top Right: Electricity vs GDP ---
fig.add_trace(go.Scatter(
    x=df1_energy["GDP_Real"], y=df1_energy["TotElectricity_2Used"],
    mode="markers+lines", name="Electricity", marker=dict(color="orange")
), row=1, col=2)

# --- Bottom Left: Energy vs Population ---
fig.add_trace(go.Scatter(
    x=df1_energy["POP_US"], y=df1_energy["TotEnergy_2EndUse"],
    mode="markers+lines", name="Total Energy", marker=dict(color="blue"), showlegend=False
), row=2, col=1)
# --- Bottom Right: Electricity vs Population ---
fig.add_trace(go.Scatter(
    x=df1_energy["POP_US"], y=df1_energy["TotElectricity_2Used"],
    mode="markers+lines", name="Electricity", marker=dict(color="orange"), showlegend=False
), row=2, col=2)

# Layout
fig.update_layout(
    title=dict(
        text="U.S. Energy & Electricity vs GDP and Population (1949–2024)",
        x=0.5, xanchor="center"
    ),
    template="plotly_white",
    width=1000, height=600,
    legend=dict(
        orientation="h", y=-0.2, x=0.5, xanchor="center", yanchor="top"
    )
)

# Axis labels
fig.update_xaxes(title_text="Real GDP (Billion chained 2017 dollars)", row=2, col=1)
fig.update_yaxes(title_text="Usage (Trillion Btu)", row=1, col=1)

fig.update_xaxes(title_text="Population (Millions)", row=2, col=2)
fig.update_yaxes(title_text="Usage (Trillion Btu)", row=2, col=1)

fig.show()


In [23]:
# Feature engineering: Energy Intensity Indicators
# - Energy use per real GDP (Trillion Btu per Billion chained 2017 dollars)
# - Energy use per Capita (Million Btu per Person)

# Per GDP-real
df1_energy["EnergyUse_per_GDP"] = df1_energy["TotEnergy_2EndUse"] / df1_energy["GDP_Real"]
df1_energy["ElectricityUse_per_GDP"] = df1_energy["TotElectricity_2Used"] / df1_energy["GDP_Real"]

# # Per GDP-nominal
# df1_energy["EnergyUse_per_GDP"] = df1_energy["TotEnergy_2EndUse"] / df1_energy["GDP_Nominal"]
# df1_energy["ElectricityUse_per_GDP"] = df1_energy["TotElectricity_2Used"] / df1_energy["GDP_Nominal"]

# Per Capita
df1_energy["EnergyUse_per_capita"] = df1_energy["TotEnergy_2EndUse"] / df1_energy["POP_US"]
df1_energy["ElectricityUse_per_capita"] = df1_energy["TotElectricity_2Used"] / df1_energy["POP_US"]

In [ ]:
# Energy Intensity Indicators: 1x2 Subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Energy & Electricity per GDP", "Energy & Electricity per Capita")
)

# --- Plot 1: per GDP ---
fig.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["EnergyUse_per_GDP"],
    mode="lines", name="Energy per GDP", line=dict(color="blue")
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["ElectricityUse_per_GDP"],
    mode="lines", name="Electricity per GDP", line=dict(color="orange")
), row=1, col=1)

# --- Plot 2: per Capita ---
fig.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["EnergyUse_per_capita"],
    mode="lines", name="Energy per Capita", line=dict(color="green")
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["ElectricityUse_per_capita"],
    mode="lines", name="Electricity per Capita", line=dict(color="red")
), row=1, col=2)

# Layout
fig.update_layout(
    title=dict(text="U.S. Energy & Electricity Intensity Indicators (1949–2024)", x=0.5, xanchor="center"),
    template="plotly_white",
    width=1000, height=500,
    legend=dict(orientation="h", x=0.5, xanchor="center", y=-0.2)  # legend below
)

# Axis labels
fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_yaxes(title_text="Usage/GDP (Trillion Btu per $Billion)", row=1, col=1)

fig.update_xaxes(title_text="Year", row=1, col=2)
fig.update_yaxes(title_text="Usage/Capita (Million Btu per Person)", row=1, col=2)

fig.show()


In [24]:
# Energy Intensity Indicators: 2x2 Subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Total Energy Use per GDP",
        "Electricity Use per GDP",
        "Total Energy Use per Capita",
        "Electricity Use per Capita"
    )
)

# --- Top Left: Energy per GDP ---
fig.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["EnergyUse_per_GDP"],
    mode="lines", name="Energy per GDP", line=dict(color="blue")
), row=1, col=1)

# --- Top Right: Electricity per GDP ---
fig.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["ElectricityUse_per_GDP"],
    mode="lines", name="Electricity per GDP", line=dict(color="orange")
), row=1, col=2)

# --- Bottom Left: Energy per Capita ---
fig.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["EnergyUse_per_capita"],
    mode="lines", name="Energy per Capita", line=dict(color="green")
), row=2, col=1)

# --- Bottom Right: Electricity per Capita ---
fig.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["ElectricityUse_per_capita"],
    mode="lines", name="Electricity per Capita", line=dict(color="red")
), row=2, col=2)

# Layout
fig.update_layout(
    title=dict(
        text="U.S. Energy & Electricity Intensity Indicators (1949–2024)",
        x=0.5, xanchor="center"
    ),
    template="plotly_white",
    width=1000, height=600,
    showlegend=False  # hide legend since each subplot has 1 trace
)

# Axis labels
# fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_yaxes(title_text="Usage (Trillion Btu per $Billion)", row=1, col=1)

# fig.update_xaxes(title_text="Year", row=1, col=2)
# fig.update_yaxes(title_text="Usage (Trillion Btu per $Billion)", row=1, col=2)

fig.update_xaxes(title_text="Year", row=2, col=1)
fig.update_yaxes(title_text="Usage (Million Btu per Capita)", row=2, col=1)

fig.update_xaxes(title_text="Year", row=2, col=2)
# fig.update_yaxes(title_text="Usage (Million Btu per Capita)", row=2, col=2)

fig.show()


In [47]:
# - Total energy expenditures (Million nominal dollars) [change to 'Billion $']
# - Energy expenditures as share of GDP (Percent)

fig = go.Figure() 

# Bar: Total Expenditures
fig.add_trace(go.Bar(
    x=df1_energy["Year"],
    y=df1_energy["Energy_Expenditure"] / 1000, # Convert from Million to Billion dollars
    name="Total Energy Expenditures",
    marker_color="steelblue",
    yaxis="y1"
))

# Line: Expenditures as % of GDP
fig.add_trace(go.Scatter(
    x=df1_energy["Year"],
    y=df1_energy["pct_EnergyExpend_w_GDP"],
    name="Expenditures (% of GDP)",
    mode="lines+markers",
    line=dict(color="darkorange", width=3),
    marker=dict(size=6, color="darkorange"),
    yaxis="y2"
))

# Layout with secondary y-axis
fig.update_layout(
    title=dict(
        text="U.S. Energy Expenditures and Share of GDP (1970-2023)",
        x=0.5, xanchor="center"
    ),
    template="plotly_white",
    width=900, height=500,
    xaxis=dict(title="Year"),

    # Left axis (blue, bars)
    yaxis=dict(
        title=dict(text="Total Energy Expenditures (Billion $)", font=dict(color="steelblue")),
        tickfont=dict(color="steelblue"),
        side="left"
    ),

    # Right axis (orange, line)
    yaxis2=dict(
        title=dict(text="Expenditures as % of GDP", font=dict(color="chocolate")),
        tickfont=dict(color="chocolate"),
        overlaying="y",
        side="right"
    ),

    legend=dict(
        orientation="h",
        y=1.10, x=0.5, xanchor="center", yanchor="top"
    )
)

fig.show()


In [50]:
# Energy Expenditures per Capita (Nominal Dollars)

fig = px.line(
    df1_energy,
    x="Year",
    y="EnergyExpend_per_capita",
    title="U.S. Energy Expenditures per Capita (1970–2023)",
    labels={
        "EnergyExpend_per_capita": "Energy Expenditures per Capita (Nominal $)",
        "Year": "Year"
    }
)

fig.update_traces(line=dict(color="crimson", width=3))

fig.update_layout(
    template="plotly_white",
    width=900, height=500,
    title=dict(x=0.5, xanchor="center"),  # center the title
    yaxis=dict(tickprefix="$", separatethousands=True)  # add $ sign, nice formatting
)

fig.show()


In [68]:
# Average energy price 

# Calculate average energy price ($ per MMBtu - Million Btu)
# - Total energy expenditures (million nominal dollars)
# - Total end-use energy consumption (Trillion Btu)
df1_energy["avg_price_per_MMBtu"] = (df1_energy["Energy_Expenditure"] * 1e6) / (df1_energy["TotEnergy_2EndUse"] * 1e6)


# Create line plot for average energy price


In [69]:


fig = go.Figure()

# Line: Energy Expenditures per Capita
fig.add_trace(go.Scatter(
    x=df1_energy["Year"],
    y=df1_energy["EnergyExpend_per_capita"],
    mode="lines+markers",
    name="Expenditures per Capita ($)",
    line=dict(color="brown", width=3),
    marker=dict(size=6)
))

# Line: Average Energy Price ($ per Btu) on secondary y-axis
fig.add_trace(go.Scatter(
    x=df1_energy["Year"],
    y=df1_energy["avg_price_per_MMBtu"],
    mode="lines+markers",
    name="Average Energy Price ($/MMBtu)",
    line=dict(color="darkblue", width=3, dash="dash"),
    marker=dict(size=6),
    yaxis="y2"
))

# Layout
fig.update_layout(
    title=dict(
        text="U.S. Energy Expenditures per Capita and Average Energy Price (1949–2023)",
        x=0.5, xanchor="center"
    ),
    template="plotly_white",
    width=900, height=500,
    xaxis=dict(title="Year"),
    yaxis=dict(
        title=dict(text="Expenditures per Capita (Nominal $)", font=dict(color="brown")),
        tickfont=dict(color="brown")
    ),
    yaxis2=dict(
        title=dict(text="Average Energy Price ($ per MM Btu)", font=dict(color="darkblue")),
        overlaying="y",
        side="right",
        tickfont=dict(color="darkblue")
    ),
    legend=dict(
        orientation="h",
        y=1.1, x=0.5, xanchor="center", yanchor="top"
    )
)

fig.show()


In [83]:
# Heating Degree Days (HDD) and Cooling Degree Days (CDD)
fig = go.Figure()

# HDD bars
fig.add_trace(go.Bar(
    x=df1_energy["Year"],
    y=df1_energy["days_heating"],
    name="Heating Degree Days (HDD)",
    marker_color="firebrick"
))

# CDD bars
fig.add_trace(go.Bar(
    x=df1_energy["Year"],
    y=df1_energy["days_cooling"],
    name="Cooling Degree Days (CDD)",
    marker_color="royalblue"
))

fig.update_layout(
    barmode="stack",  # stacked bars
    title=dict(text="U.S. Heating and Cooling Degree Days 1949–2024", x=0.5),
    xaxis_title="Year",
    yaxis_title="Degree Days",
    template="plotly_white",
    width=950, height=500,
    legend=dict(orientation="h", x=0.5, xanchor="center", y=1.1)
)

fig.show()


In [125]:
# Relationship between Degree Days and Energy Consumption

# Create a total degree days column
df1_energy["Total_DegreeDays"] = df1_energy["days_heating"] + df1_energy["days_cooling"]

# Create subplot layout: 1 row, 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        "End-Use Energy",
        "Electricity Use"
    )
)

# Left plot: Total End-use Energy
fig.add_trace(
    go.Scatter(
        x=df1_energy["Total_DegreeDays"],
        y=df1_energy["TotEnergy_2EndUse"],
        mode="markers",
        marker=dict(color="blue", size=7, opacity=0.7),
        name="End-Use Energy",
        showlegend=False
    ),
    row=1, col=1
)

# Right plot: Electricity Use
fig.add_trace(
    go.Scatter(
        x=df1_energy["Total_DegreeDays"],
        y=df1_energy["TotElectricity_2Used"],
        mode="markers",
        marker=dict(color="green", size=7, opacity=0.7),
        name="Electricity Use",
        showlegend=False
    ),
    row=1, col=2
)

# Layout
fig.update_layout(
    title=dict(text="End-Use Energy & Electricity Use vs. Total Degree Days",
        x=0.5),
    template="plotly_white",
    width=1000, height=450
)

# Axes labels
fig.update_xaxes(title="Total Degree Days", row=1, col=1)
fig.update_yaxes(title="Energy Use (Trillion Btu)", row=1, col=1)

fig.update_xaxes(title="Total Degree Days", row=1, col=2)
# fig.update_yaxes(title="Electricity Use (Trillion Btu)", row=1, col=2)

fig.show()


In [ ]:
# Plot energy use and degree days over years

# Create subplot grid: 1 row x 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("End-Use Energy", "Electricity Use"),
    specs=[[{"secondary_y": True}, {"secondary_y": True}]]
)

# ==== Left subplot: End-Use Energy vs Degree Days ====
fig.add_trace(
    go.Scatter(
        x=df1_energy["Year"],
        y=df1_energy["TotEnergy_2EndUse"],
        mode="lines+markers",
        name="End-Use Energy",
        line=dict(color="blue")
    ),
    row=1, col=1, secondary_y=False
)

fig.add_trace(
    go.Scatter(
        x=df1_energy["Year"],
        y=df1_energy["Total_DegreeDays"],
        mode="lines+markers",
        name="Total Degree Days",
        line=dict(color="red", dash="dot"),
        showlegend=False
    ),
    row=1, col=1, secondary_y=True
)

# ==== Right subplot: Electricity Use vs Degree Days ====
fig.add_trace(
    go.Scatter(
        x=df1_energy["Year"],
        y=df1_energy["TotElectricity_2Used"],
        mode="lines+markers",
        name="Electricity Use",
        line=dict(color="green")
    ),
    row=1, col=2, secondary_y=False
)

fig.add_trace(
    go.Scatter(
        x=df1_energy["Year"],
        y=df1_energy["Total_DegreeDays"],
        mode="lines+markers",
        name="Total Degree Days",
        line=dict(color="red", dash="dot")  # dashed red so it stands out
    ),
    row=1, col=2, secondary_y=True
)

# Layout
fig.update_layout(
    title=dict(
        text="Energy & Electricity Use vs Total Degree Days Over Time (1949–2024)",
        x=0.5
    ),
    template="plotly_white",
    width=1000, height=450,
    legend=dict(orientation="h", x=0.5, xanchor="center", y=-0.2)
)

# Update axis titles
fig.update_yaxes(title_text="Energy Use (Trillion Btu)", row=1, col=1, secondary_y=False)
# fig.update_yaxes(title_text="Total Degree Days", row=1, col=1, secondary_y=True)

# fig.update_yaxes(title_text="Electricity Use (Trillion Btu)", row=1, col=2, secondary_y=False)
fig.update_yaxes(title_text="Total Degree Days", row=1, col=2, secondary_y=True)

fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=2)

fig.show()


#### Predictive regression analysis
Dependent variable (DV): Total end-use energy consumption; Total Electricity use

Predictors (IV): GDP, Population, and total degree days

In [ ]:
# Correlation matrix 
corr_matrix = df1_energy[["TotEnergy_2EndUse", "TotElectricity_2Used", "GDP_Real", 
    "POP_US", "Total_DegreeDays"]].corr()
print(corr_matrix)

# Note: R of between GDP and population is 0.985, very high, indicating potential multicollinearity.
# Both GDP and population also have a high correlation with total energy/electricity consumptions, 
# which could confound analyses.

# Heatmap of correlation matrix
fig = px.imshow(
    corr_matrix,
    text_auto='.3f',
    aspect="auto",
    color_continuous_scale=px.colors.diverging.RdBu,
    title="Correlation Matrix Heatmap"
)
fig.show()

                      TotEnergy_2EndUse  TotElectricity_2Used  GDP_Real    POP_US  Total_DegreeDays
TotEnergy_2EndUse              1.000000              0.959495  0.872774  0.930697         -0.684371
TotElectricity_2Used           0.959495              1.000000  0.952358  0.984696         -0.778160
GDP_Real                       0.872774              0.952358  1.000000  0.985212         -0.787429
POP_US                         0.930697              0.984696  0.985212  1.000000         -0.779367
Total_DegreeDays              -0.684371             -0.778160 -0.787429 -0.779367          1.000000


In [ ]:
# Check Variance Inflation Factor (VIF): multicollinearity among predictors
# VIF > 5-10 indicates high multicollinearity 
# The VIFs for GDP and population are 35.2 and 34.1, respectively, indicating severe multicollinearity.

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

X = add_constant(df1_energy[["GDP_Real", "POP_US", "Total_DegreeDays"]])
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)

            Feature          VIF
0             const  1362.875691
1          GDP_Real    35.236421
2            POP_US    34.102632
3  Total_DegreeDays     2.634921


##### Regression model: 1949-2024

In [ ]:
# 1-1. Select independent and dependent variables
X = df1_energy[["GDP_Real", "POP_US", "Total_DegreeDays"]]

# Add constant for intercept
# import statsmodels.api as sm
X = sm.add_constant(X)

In [ ]:
# Dependent variable: Total end-use energy consumption
y_energy = df1_energy["TotEnergy_2EndUse"]

# Fit model
model_energy = sm.OLS(y_energy, X).fit()

# Summary
print("Regression: Total Energy Consumption ~ GDP + Population + Total Degree Days")
print(model_energy.summary())

# Note: The regression coefficient for GDP_Real is 'negative' (-3.3267). However, the scatter plot shows 
# a 'positive' relationship, suggesting potential issues with multicollinearity or model specification.

Regression: Total Energy Consumption ~ GDP + Population + Total Degree Days
                            OLS Regression Results                            
Dep. Variable:      TotEnergy_2EndUse   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                     334.1
Date:                Thu, 21 Aug 2025   Prob (F-statistic):           3.67e-42
Time:                        08:56:19   Log-Likelihood:                -729.48
No. Observations:                  76   AIC:                             1467.
Df Residuals:                      72   BIC:                             1476.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------

In [ ]:
# 1-2. Only GDP as IV. The regression coefficient for GDP_Real becomes 'positive' (1.9565), 
# aligning with the scatter plot's indication of a 'positive' relationship.

X = df1_energy[["GDP_Real"]]
X = sm.add_constant(X)
y = df1_energy["TotEnergy_2EndUse"]
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      TotEnergy_2EndUse   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.759
Method:                 Least Squares   F-statistic:                     236.6
Date:                Sun, 24 Aug 2025   Prob (F-statistic):           9.43e-25
Time:                        04:16:55   Log-Likelihood:                -777.68
No. Observations:                  76   AIC:                             1559.
Df Residuals:                      74   BIC:                             1564.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.907e+04   1531.830     25.507      0.0

In [ ]:
# 1-3. IV = Population (POP_US); DV = TotEnergy_2EndUse

X = df1_energy[["POP_US"]]
X = sm.add_constant(X)
y = df1_energy["TotEnergy_2EndUse"]
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      TotEnergy_2EndUse   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.864
Method:                 Least Squares   F-statistic:                     479.1
Date:                Sun, 24 Aug 2025   Prob (F-statistic):           4.74e-34
Time:                        07:47:31   Log-Likelihood:                -755.75
No. Observations:                  76   AIC:                             1516.
Df Residuals:                      74   BIC:                             1520.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2989.8367   2640.102      1.132      0.2

In [ ]:
# 2-1. IV = GDP + Population + Total Degree Days; DV = Total Electricity use
y_electricity = df1_energy["TotElectricity_2Used"]

# Fit model
model_elec = sm.OLS(y_electricity, X).fit()

# Summary
print("Regression: Total Electricity Use ~ GDP + Population + Total Degree Days")
print(model_elec.summary())


Regression: Total Electricity Use ~ GDP + Population + Total Degree Days
                             OLS Regression Results                             
Dep. Variable:     TotElectricity_2Used   R-squared:                       0.982
Model:                              OLS   Adj. R-squared:                  0.981
Method:                   Least Squares   F-statistic:                     1290.
Date:                  Thu, 21 Aug 2025   Prob (F-statistic):           1.78e-62
Time:                          08:58:36   Log-Likelihood:                -589.85
No. Observations:                    76   AIC:                             1188.
Df Residuals:                        72   BIC:                             1197.
Df Model:                             3                                         
Covariance Type:              nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

In [ ]:
# 2-2. IV = GDP_Real; DV = TotElectricity_2Used

X = df1_energy[["GDP_Real"]]
X = sm.add_constant(X)
y = df1_energy["TotElectricity_2Used"]
model = sm.OLS(y, X).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     TotElectricity_2Used   R-squared:                       0.907
Model:                              OLS   Adj. R-squared:                  0.906
Method:                   Least Squares   F-statistic:                     721.6
Date:                  Sun, 24 Aug 2025   Prob (F-statistic):           6.65e-40
Time:                          08:01:45   Log-Likelihood:                -651.70
No. Observations:                    76   AIC:                             1307.
Df Residuals:                        74   BIC:                             1312.
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1228.6182    291.963

In [ ]:
# 2-3. IV = POP_US; DV = TotElectricity_2Used

X = df1_energy[["POP_US"]]
X = sm.add_constant(X)
y = df1_energy["TotElectricity_2Used"]
model = sm.OLS(y, X).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     TotElectricity_2Used   R-squared:                       0.970
Model:                              OLS   Adj. R-squared:                  0.969
Method:                   Least Squares   F-statistic:                     2362.
Date:                  Sun, 24 Aug 2025   Prob (F-statistic):           6.68e-58
Time:                          08:01:52   Log-Likelihood:                -609.17
No. Observations:                    76   AIC:                             1222.
Df Residuals:                        74   BIC:                             1227.
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.021e+04    383.718

In [ ]:
# Plot Predicted vs Actual Values

# Create subplot grid: 1 row, 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Total Energy Consumption", "Total Electricity Use")
)

# --- Left: Total Energy ---
fig.add_trace(go.Scatter(
    x=df1_energy["TotEnergy_2EndUse"],
    y=df1_energy["Pred_Energy"],
    mode="markers",
    name="Energy"
), row=1, col=1)

# Add 45-degree reference line
fig.add_trace(go.Scatter(
    x=[df1_energy["TotEnergy_2EndUse"].min(), df1_energy["TotEnergy_2EndUse"].max()],
    y=[df1_energy["TotEnergy_2EndUse"].min(), df1_energy["TotEnergy_2EndUse"].max()],
    mode="lines",
    line=dict(color="red", dash="dash"),
    showlegend=False
), row=1, col=1)

# --- Right: Total Electricity ---
fig.add_trace(go.Scatter(
    x=df1_energy["TotElectricity_2Used"],
    y=df1_energy["Pred_Electricity"],
    mode="markers",
    name="Electricity"
), row=1, col=2)

# Add 45-degree reference line
fig.add_trace(go.Scatter(
    x=[df1_energy["TotElectricity_2Used"].min(), df1_energy["TotElectricity_2Used"].max()],
    y=[df1_energy["TotElectricity_2Used"].min(), df1_energy["TotElectricity_2Used"].max()],
    mode="lines",
    line=dict(color="red", dash="dash"),
    showlegend=False
), row=1, col=2)

# Layout
fig.update_layout(
    title=dict(text="Predicted vs Actual: Energy & Electricity Consumptions", x=0.5),
    template="plotly_white",
    width=1000, height=500,
    xaxis1=dict(title="Actual Use (Trillion Btu)"),
    yaxis1=dict(title="Predicted Use (Trillion Btu)"),
    xaxis2=dict(title="Actual Use (Trillion Btu)"),
    # yaxis2=dict(title="Predicted Use (Trillion Btu)"),
    showlegend=False
)

fig.show()


In [ ]:
# Plot Predicted vs Actual Values: add regression formulas and R-squared

# --- Regression: Energy ---
X_energy = df1_energy[["GDP_Real", "POP_US", "Total_DegreeDays"]]  # predictors
X_energy = sm.add_constant(X_energy)  # add intercept
y_energy = df1_energy["TotEnergy_2EndUse"]

model_energy = sm.OLS(y_energy, X_energy).fit()
df1_energy["Pred_Energy"] = model_energy.predict(X_energy)

# --- Regression: Electricity ---
X_elec = df1_energy[["GDP_Real", "POP_US", "Total_DegreeDays"]]
X_elec = sm.add_constant(X_elec)
y_elec = df1_energy["TotElectricity_2Used"]

model_elec = sm.OLS(y_elec, X_elec).fit()
df1_energy["Pred_Electricity"] = model_elec.predict(X_elec)

# Extract formulas + R^2
eqn_energy = (
    f"Energy = {model_energy.params[0]:.2f} + {model_energy.params[1]:.2f}·GDP + "
    f"{model_energy.params[2]:.2f}·Pop + {model_energy.params[3]:.2f}·DegreeDays"
)
r2_energy = f"R² = {model_energy.rsquared:.3f}"

eqn_elec = (
    f"Electricity = {model_elec.params[0]:.2f} + {model_elec.params[1]:.2f}·GDP + "
    f"{model_elec.params[2]:.2f}·Pop + {model_elec.params[3]:.2f}·DegreeDays"
)
r2_elec = f"R² = {model_elec.rsquared:.3f}"

# --- PLOTS ---
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Total Energy Consumption", "Total Electricity Use")
)

# Left: Energy
fig.add_trace(go.Scatter(
    x=y_energy, y=df1_energy["Pred_Energy"],
    mode="markers", name="Energy",
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=[y_energy.min(), y_energy.max()],
    y=[y_energy.min(), y_energy.max()],
    mode="lines", line=dict(color="red", dash="dash"),
    showlegend=False
), row=1, col=1)

fig.add_annotation(
    text=f"{eqn_energy}<br>{r2_energy}",
    xref="x domain", yref="y domain",
    x=0.05, y=0.95, showarrow=False,
    align="left", font=dict(size=10, family="Arial", color="brown", weight="bold"),
    row=1, col=1
)

# Right: Electricity
fig.add_trace(go.Scatter(
    x=y_elec, y=df1_energy["Pred_Electricity"],
    mode="markers", name="Electricity",
    showlegend=False
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=[y_elec.min(), y_elec.max()],
    y=[y_elec.min(), y_elec.max()],
    mode="lines", line=dict(color="red", dash="dash"),
    showlegend=False
), row=1, col=2)

fig.add_annotation(
    text=f"{eqn_elec}<br>{r2_elec}",
    xref="x domain", yref="y domain",
    x=0.05, y=0.95, showarrow=False,
    align="left", font=dict(size=10, family="Arial", color="brown", weight="bold"),
    row=1, col=2
)

# Layout
fig.update_layout(
    title=dict(text="Predicted vs Actual: Energy & Electricity Consumptions", x=0.5),
    template="plotly_white",
    width=1100, height=500,
    xaxis1=dict(title="Actual Use (Trillion Btu)"),
    yaxis1=dict(title="Predicted Use (Trillion Btu)"),
    xaxis2=dict(title="Actual Use (Trillion Btu)"),
    # yaxis2=dict(title="Predicted Use (Trillion Btu)")
    # legend=dict(
    #     orientation="h",   # horizontal legend
    #     yanchor="bottom", 
    #     y=1,             # move above the plots
    #     xanchor="center", 
    #     x=0.5)
)

fig.show()


C:\Users\lm\AppData\Local\Temp\ipykernel_20412\3634263794.py:21: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\lm\AppData\Local\Temp\ipykernel_20412\3634263794.py:22: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\lm\AppData\Local\Temp\ipykernel_20412\3634263794.py:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\lm\AppData\Local\Temp\ipykernel_20412\3634263794.py:28: FutureWarning:

Series.__getitem__ tre

In [132]:
# Plot Predicted vs Actual Values: add regression formulas and R-squared
# Exclude Total Degree Days from predictors

# --- Regression: Energy ---
X_energy = df1_energy[["GDP_Real", "POP_US"]]  # predictors
X_energy = sm.add_constant(X_energy)  # add intercept
y_energy = df1_energy["TotEnergy_2EndUse"]

model_energy = sm.OLS(y_energy, X_energy).fit()
df1_energy["Pred_Energy"] = model_energy.predict(X_energy)

# --- Regression: Electricity ---
X_elec = df1_energy[["GDP_Real", "POP_US"]]
X_elec = sm.add_constant(X_elec)
y_elec = df1_energy["TotElectricity_2Used"]

model_elec = sm.OLS(y_elec, X_elec).fit()
df1_energy["Pred_Electricity"] = model_elec.predict(X_elec)

# Extract formulas + R^2
eqn_energy = (
    f"Energy = {model_energy.params[0]:.2f} + {model_energy.params[1]:.2f}·GDP + "
    f"{model_energy.params[2]:.2f}·Pop"
)
r2_energy = f"R² = {model_energy.rsquared:.3f}"

eqn_elec = (
    f"Electricity = {model_elec.params[0]:.2f} + {model_elec.params[1]:.2f}·GDP + "
    f"{model_elec.params[2]:.2f}·Pop"
)
r2_elec = f"R² = {model_elec.rsquared:.3f}"

# --- PLOTS ---
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Total Energy Consumption", "Total Electricity Use")
)

# Left: Energy
fig.add_trace(go.Scatter(
    x=y_energy, y=df1_energy["Pred_Energy"],
    mode="markers", name="Energy",
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=[y_energy.min(), y_energy.max()],
    y=[y_energy.min(), y_energy.max()],
    mode="lines", line=dict(color="red", dash="dash"),
    showlegend=False
), row=1, col=1)

fig.add_annotation(
    text=f"{eqn_energy}<br>{r2_energy}",
    xref="x domain", yref="y domain",
    x=0.05, y=0.95, showarrow=False,
    align="left", font=dict(size=10, family="Arial", color="brown", weight="bold"),
    row=1, col=1
)

# Right: Electricity
fig.add_trace(go.Scatter(
    x=y_elec, y=df1_energy["Pred_Electricity"],
    mode="markers", name="Electricity",
    showlegend=False
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=[y_elec.min(), y_elec.max()],
    y=[y_elec.min(), y_elec.max()],
    mode="lines", line=dict(color="red", dash="dash"),
    showlegend=False
), row=1, col=2)

fig.add_annotation(
    text=f"{eqn_elec}<br>{r2_elec}",
    xref="x domain", yref="y domain",
    x=0.05, y=0.95, showarrow=False,
    align="left", font=dict(size=10, family="Arial", color="brown", weight="bold"),
    row=1, col=2
)

# Layout
fig.update_layout(
    title=dict(text="Predicted vs Actual: Energy & Electricity Consumptions", x=0.5),
    template="plotly_white",
    width=1100, height=500,
    xaxis1=dict(title="Actual Use (Trillion Btu)"),
    yaxis1=dict(title="Predicted Use (Trillion Btu)"),
    xaxis2=dict(title="Actual Use (Trillion Btu)"),
    # yaxis2=dict(title="Predicted Use (Trillion Btu)")
    # legend=dict(
    #     orientation="h",   # horizontal legend
    #     yanchor="bottom", 
    #     y=1,             # move above the plots
    #     xanchor="center", 
    #     x=0.5)
)

fig.show()

C:\Users\lm\AppData\Local\Temp\ipykernel_20412\2593497436.py:22: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\lm\AppData\Local\Temp\ipykernel_20412\2593497436.py:23: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\lm\AppData\Local\Temp\ipykernel_20412\2593497436.py:28: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\lm\AppData\Local\Temp\ipykernel_20412\2593497436.py:29: FutureWarning:

Series.__getitem__ tre

##### Forcasting energy/electricity use in 2025-2030
- Use the regression model based on data of 1949-2024

In [19]:
# Predicting Total Energy Consumption and Total Electricity Use through 2025-2030
# - According to EIA, real GDP will grow by 2.0% annually from 2024 to 2030.
# - Since 2000, the U.S. population has grown by approximately 0.8% annually (U.S. Census Bureau).

# Step 1. Extend GDP & Population with growth rates

# Take last known year, GDP, and Population
last_year = df1_energy["Year"].max()
last_gdp = df1_energy.loc[df1_energy["Year"] == last_year, "GDP_Real"].values[0]
last_pop = df1_energy.loc[df1_energy["Year"] == last_year, "POP_US"].values[0]

# Forecast years
future_years = np.arange(2025, 2031)

# Project GDP and Population
# - Real GDP grows 2.0% per year after 2024
# - Population grows 0.8% per year after 2024
future_gdp = [last_gdp * ((1.02) ** (year - last_year)) for year in future_years]
future_pop = [last_pop * ((1.008) ** (year - last_year)) for year in future_years]

df_future = pd.DataFrame({
    "Year": future_years,
    "GDP_Real": future_gdp,
    "POP_US": future_pop
})

In [134]:
# Step 2. Fit Linear Regression Models (using historical data)
from sklearn.linear_model import LinearRegression

# Features and targets
X = df1_energy[["GDP_Real", "POP_US"]]
y_energy = df1_energy["TotEnergy_2EndUse"]
y_elec = df1_energy["TotElectricity_2Used"]

# Fit models
model_energy = LinearRegression().fit(X, y_energy)
model_elec = LinearRegression().fit(X, y_elec)

In [ ]:
# Step 3. Predict for 2025–2030; IVs = GDP_Real, POP_US

X_future = df_future[["GDP_Real", "POP_US"]]
df_future["Pred_Energy"] = model_energy.predict(X_future)
df_future["Pred_Electricity"] = model_elec.predict(X_future)

# Combine with historical data for plotting
df_plot = pd.concat([df1_energy[["Year","TotEnergy_2EndUse","TotElectricity_2Used"]], df_future], ignore_index=True)

In [140]:
# Step 4. Plot actual vs forecast (2025–2030 shaded)

fig = go.Figure()

# Historical Energy
fig.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["TotEnergy_2EndUse"],
    mode="lines+markers", name="Actual Energy Use", line=dict(color="blue")
))

# Forecast Energy
fig.add_trace(go.Scatter(
    x=df_future["Year"], y=df_future["Pred_Energy"],
    mode="lines+markers", name="Forecast Energy Use", line=dict(color="maroon", dash="dash")
))

# Historical Electricity
fig.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["TotElectricity_2Used"],
    mode="lines+markers", name="Actual Electricity Use", line=dict(color="green")
))

# Forecast Electricity
fig.add_trace(go.Scatter(
    x=df_future["Year"], y=df_future["Pred_Electricity"],
    mode="lines+markers", name="Forecast Electricity Use", line=dict(color="orange", dash="dash")
))

# Layout
fig.update_layout(
    title=dict(
        text="Predicted U.S. Energy & Electricity Use (2025–2030)",
        x=0.5
    ),
    template="plotly_white",
    xaxis_title="Year",
    yaxis_title="Energy Use (Trillion Btu)",
    width=900, height=500,
    legend=dict(orientation="h", x=0.5, xanchor="center", y=1.1)
)

fig.show()


In [21]:
# Only one IV = POP_US
# Step 2. Fit Linear Regression Models (using historical data)

df_future2 = pd.DataFrame({
    "Year": future_years,
        "POP_US": future_pop
})

# Features and targets
X = df1_energy[["POP_US"]]
y_energy = df1_energy["TotEnergy_2EndUse"]
y_elec = df1_energy["TotElectricity_2Used"]

# Fit models
model_energy = LinearRegression().fit(X, y_energy)
model_elec = LinearRegression().fit(X, y_elec)

In [22]:
# Step 3-2. Predict for 2025–2030; IVs = POP_US

X_future2 = df_future2[["POP_US"]]
df_future2["Pred_Energy"] = model_energy.predict(X_future2)
df_future2["Pred_Electricity"] = model_elec.predict(X_future2)

# Combine with historical data for plotting
df_plot = pd.concat([df1_energy[["Year","TotEnergy_2EndUse","TotElectricity_2Used"]], df_future2], ignore_index=True)

In [ ]:
# Step 4-2. Plot actual vs forecast (2025–2030 shaded)
# The following plot (IV = POP_US) shows higher energy/electricity use forecasts. This looks
# more reasonable than the above plot (IV = GDP_Real and POP_US). 

fig = go.Figure()

# Historical Energy
fig.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["TotEnergy_2EndUse"],
    mode="lines+markers", name="Actual Energy Use", line=dict(color="blue")
))

# Forecast Energy
fig.add_trace(go.Scatter(
    x=df_future2["Year"], y=df_future2["Pred_Energy"],
    mode="lines+markers", name="Forecast Energy Use", line=dict(color="maroon", dash="dash")
))

# Historical Electricity
fig.add_trace(go.Scatter(
    x=df1_energy["Year"], y=df1_energy["TotElectricity_2Used"],
    mode="lines+markers", name="Actual Electricity Use", line=dict(color="green")
))

# Forecast Electricity
fig.add_trace(go.Scatter(
    x=df_future2["Year"], y=df_future2["Pred_Electricity"],
    mode="lines+markers", name="Forecast Electricity Use", line=dict(color="orange", dash="dash")
))

# Layout
fig.update_layout(
    title=dict(
        text="Predicted U.S. Energy & Electricity Use (2025–2030)",
        x=0.5
    ),
    template="plotly_white",
    xaxis_title="Year",
    yaxis_title="Energy Use (Trillion Btu)",
    width=900, height=500,
    legend=dict(orientation="h", x=0.5, xanchor="center", y=1.1)
)

fig.show()


##### Forcasting energy/electricity use in 2015-2024
- Use the regression model based on data of 1949-2014

In [ ]:
# Model 1: IV = Population

# 1. Filter historical data to 1929–2014
df_hist = df1_energy[(df1_energy["Year"] >= 1929) & (df1_energy["Year"] <= 2014)]
df_future = df1_energy[(df1_energy["Year"] >= 2015) & (df1_energy["Year"] <= 2025)]


# 2. Fit regression models (Population → Energy, Electricity)
# Features and targets
X = df_hist[["POP_US"]]
y_energy = df_hist["TotEnergy_2EndUse"]
y_elec = df_hist["TotElectricity_2Used"]

# Fit models
model_energy = LinearRegression().fit(X, y_energy)
model_elec = LinearRegression().fit(X, y_elec)

# Predict for 2015–2025
df_future["Pred_Energy"] = model_energy.predict(df_future[["POP_US"]])
df_future["Pred_Electricity"] = model_elec.predict(df_future[["POP_US"]])

C:\Users\lm\AppData\Local\Temp\ipykernel_19724\984928815.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\lm\AppData\Local\Temp\ipykernel_19724\984928815.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
# 3. Combine actual + forecast
df_plot = pd.concat([
    df_hist[["Year","TotEnergy_2EndUse","TotElectricity_2Used"]],
    df_future[["Year","TotEnergy_2EndUse","TotElectricity_2Used","Pred_Energy","Pred_Electricity"]]
], ignore_index=True)

# 4. Plot actual vs predicted (2015–2025 forecasts)
import plotly.graph_objects as go

fig = go.Figure()

# Energy Actual
fig.add_trace(go.Scatter(
    x=df_plot["Year"], y=df_plot["TotEnergy_2EndUse"],
    mode="lines+markers", name="Actual Energy Use", line=dict(color="blue")
))

# Energy Predicted
fig.add_trace(go.Scatter(
    x=df_plot["Year"], y=df_plot["Pred_Energy"],
    mode="lines+markers", name="Predicted Energy Use", line=dict(color="brown", dash="dash")
))

# Electricity Actual
fig.add_trace(go.Scatter(
    x=df_plot["Year"], y=df_plot["TotElectricity_2Used"],
    mode="lines+markers", name="Actual Electricity Use", line=dict(color="green")
))

# Electricity Predicted
fig.add_trace(go.Scatter(
    x=df_plot["Year"], y=df_plot["Pred_Electricity"],
    mode="lines+markers", name="Predicted Electricity Use", line=dict(color="orange", dash="dash")
))

# Layout
fig.update_layout(
    title=dict(text="Forecast of U.S. Energy & Electricity Use (2015–2025) based on Population", x=0.5),
    template="plotly_white",
    xaxis_title="Year",
    yaxis_title="Trillion Btu",
    legend=dict(orientation="h", x=0.5, xanchor="center", y=-0.2)
)

fig.show()

In [28]:
# Model 2: IV = GDP_Real

# 1. Filter historical data to 1929–2014
df_hist = df1_energy[(df1_energy["Year"] >= 1929) & (df1_energy["Year"] <= 2014)]
df_future = df1_energy[(df1_energy["Year"] >= 2015) & (df1_energy["Year"] <= 2025)]


# 2. Fit regression models (GDP_Real → Energy, Electricity)
# Features and targets
X = df_hist[["GDP_Real"]]
y_energy = df_hist["TotEnergy_2EndUse"]
y_elec = df_hist["TotElectricity_2Used"]

# Fit models
model_energy = LinearRegression().fit(X, y_energy)
model_elec = LinearRegression().fit(X, y_elec)

# Predict for 2015–2025
df_future["Pred_Energy"] = model_energy.predict(df_future[["GDP_Real"]])
df_future["Pred_Electricity"] = model_elec.predict(df_future[["GDP_Real"]])

C:\Users\lm\AppData\Local\Temp\ipykernel_19724\3700569770.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\lm\AppData\Local\Temp\ipykernel_19724\3700569770.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
# 3. Combine actual + forecast
df_plot = pd.concat([
    df_hist[["Year","TotEnergy_2EndUse","TotElectricity_2Used"]],
    df_future[["Year","TotEnergy_2EndUse","TotElectricity_2Used","Pred_Energy","Pred_Electricity"]]
], ignore_index=True)

# 4. Plot actual vs predicted (2015–2025 forecasts)
import plotly.graph_objects as go

fig = go.Figure()

# Energy Actual
fig.add_trace(go.Scatter(
    x=df_plot["Year"], y=df_plot["TotEnergy_2EndUse"],
    mode="lines+markers", name="Actual Energy Use", line=dict(color="blue")
))

# Energy Predicted
fig.add_trace(go.Scatter(
    x=df_plot["Year"], y=df_plot["Pred_Energy"],
    mode="lines+markers", name="Predicted Energy Use", line=dict(color="brown", dash="dash")
))

# Electricity Actual
fig.add_trace(go.Scatter(
    x=df_plot["Year"], y=df_plot["TotElectricity_2Used"],
    mode="lines+markers", name="Actual Electricity Use", line=dict(color="green")
))

# Electricity Predicted
fig.add_trace(go.Scatter(
    x=df_plot["Year"], y=df_plot["Pred_Electricity"],
    mode="lines+markers", name="Predicted Electricity Use", line=dict(color="orange", dash="dash")
))

# Layout
fig.update_layout(
    title=dict(text="Forecast of U.S. Energy & Electricity Use (2015–2025) based on Population", x=0.5),
    template="plotly_white",
    xaxis_title="Year",
    yaxis_title="Trillion Btu",
    legend=dict(orientation="h", x=0.5, xanchor="center", y=-0.2)
)

fig.show()

In [30]:
# Model 3: IV = GDP_Real + Population

# 1. Filter historical data to 1929–2014
df_hist = df1_energy[(df1_energy["Year"] >= 1929) & (df1_energy["Year"] <= 2014)]
df_future = df1_energy[(df1_energy["Year"] >= 2015) & (df1_energy["Year"] <= 2025)]


# 2. Fit regression models (GDP_Real → Energy, Electricity)
# Features and targets
X = df_hist[["GDP_Real", "POP_US"]]
y_energy = df_hist["TotEnergy_2EndUse"]
y_elec = df_hist["TotElectricity_2Used"]

# Fit models
model_energy = LinearRegression().fit(X, y_energy)
model_elec = LinearRegression().fit(X, y_elec)

# Predict for 2015–2025
df_future["Pred_Energy"] = model_energy.predict(df_future[["GDP_Real","POP_US"]])
df_future["Pred_Electricity"] = model_elec.predict(df_future[["GDP_Real","POP_US"]])

C:\Users\lm\AppData\Local\Temp\ipykernel_19724\3366890157.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\lm\AppData\Local\Temp\ipykernel_19724\3366890157.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
# 3. Combine actual + forecast
df_plot = pd.concat([
    df_hist[["Year","TotEnergy_2EndUse","TotElectricity_2Used"]],
    df_future[["Year","TotEnergy_2EndUse","TotElectricity_2Used","Pred_Energy","Pred_Electricity"]]
], ignore_index=True)

# 4. Plot actual vs predicted (2015–2025 forecasts)
import plotly.graph_objects as go

fig = go.Figure()

# Energy Actual
fig.add_trace(go.Scatter(
    x=df_plot["Year"], y=df_plot["TotEnergy_2EndUse"],
    mode="lines+markers", name="Actual Energy Use", line=dict(color="blue")
))

# Energy Predicted
fig.add_trace(go.Scatter(
    x=df_plot["Year"], y=df_plot["Pred_Energy"],
    mode="lines+markers", name="Predicted Energy Use", line=dict(color="brown", dash="dash")
))

# Electricity Actual
fig.add_trace(go.Scatter(
    x=df_plot["Year"], y=df_plot["TotElectricity_2Used"],
    mode="lines+markers", name="Actual Electricity Use", line=dict(color="green")
))

# Electricity Predicted
fig.add_trace(go.Scatter(
    x=df_plot["Year"], y=df_plot["Pred_Electricity"],
    mode="lines+markers", name="Predicted Electricity Use", line=dict(color="orange", dash="dash")
))

# Layout
fig.update_layout(
    title=dict(text="Forecast of U.S. Energy & Electricity Use (2015–2025) based on Population", x=0.5),
    template="plotly_white",
    xaxis_title="Year",
    yaxis_title="Trillion Btu",
    legend=dict(orientation="h", x=0.5, xanchor="center", y=-0.2)
)

fig.show()